# 🎌 tetukuri-srt-ja-zh-AItoolforlivesub
**直播字幕手搓轴-AI翻校工具** · Faster-Whisper × GPT × 弹幕校对

| 步骤 | 功能 |
|------|------|
| Step 1 | 基于参考 SRT 时间轴逐段切片 → Whisper large-v3 重新识别 |
| Step 2 | 清洗 YouTube 弹幕 JSON，过滤低质量内容 |
| Step 3 | GPT 结合弹幕上下文对 ASR 结果智能校对 |
| Step 4 | 并发调用 GPT 将日语逐行翻译为中文 |

> **运行前**：Runtime → Change runtime type → **T4 GPU**

In [ ]:
# ─────────────────────────────────────────────
# Cell 1 · 安装依赖（仅需运行一次）
# ─────────────────────────────────────────────
!apt-get update -qq && apt-get install -y ffmpeg
!pip install -q faster-whisper openai rapidfuzz tenacity tqdm

In [ ]:
# ─────────────────────────────────────────────
# Cell 2 · 拉取项目代码
# ─────────────────────────────────────────────
!git clone https://github.com/ushi-C/tetukuri-srt-ja-zh-AItoolforlivesub.git
%cd tetukuri-srt-ja-zh-AItoolforlivesub

import sys
sys.path.insert(0, 'src')

In [ ]:
# ─────────────────────────────────────────────
# Cell 3 · 初始化客户端 & 填写直播背景
# ─────────────────────────────────────────────
from getpass import getpass
from openai import OpenAI
from pipeline import (
    run_asr_from_srt, run_danmaku_cleaning,
    run_smart_proofread, run_parallel_translation,
    write_bilingual_srt, usage_stats
)

OPENAI_BASE_URL = "https://api.chatanywhere.tech/v1"  # 可替换为官方 https://api.openai.com/v1
MODEL_NAME      = "gpt-4o"                            # 或 gpt-3.5-turbo

api_key = getpass("🔑 请输入 API Key: ").strip()
client  = OpenAI(api_key=api_key, base_url=OPENAI_BASE_URL)
client._model = MODEL_NAME   # 挂载模型名供 pipeline 使用

print("=== 直播背景信息 ===")
host_info    = input("1. 配信者/所属/ファンネーム（例：ななしいんく所属 涼海ネモ）: ").strip()
stream_title = input("2. 直播标题（例：雑談）: ").strip()
my_bg = {
    "host_info":    host_info    or "未知信息",
    "stream_title": stream_title or "通用直播",
}

In [ ]:
# ─────────────────────────────────────────────
# Cell 4 · 上传文件
# ─────────────────────────────────────────────
from google.colab import files
from pathlib import Path

print("📤 上传音频文件（mp3 / wav / m4a / flac）:")
uploaded_audio = files.upload()
audio_file = next((fn for fn in uploaded_audio if fn.lower().endswith((".mp3",".wav",".m4a",".flac"))), None)
if not audio_file: raise ValueError("❌ 未检测到有效音频")

print("📤 上传参考 SRT 时间轴（.srt）:")
uploaded_srt = files.upload()
srt_file = next((fn for fn in uploaded_srt if fn.lower().endswith(".srt")), None)
if not srt_file: raise ValueError("❌ 未检测到有效 SRT")

print("📤 是否上传弹幕 JSON？(yes/no)")
dm_path = None
if input().strip().lower() == "yes":
    uploaded_dm = files.upload()
    dm_path = list(uploaded_dm.keys())[0] if uploaded_dm else None

print(f"\n🎵 音频: {audio_file}\n📝 SRT : {srt_file}\n💬 弹幕: {dm_path or '（跳过）'}")

In [ ]:
# ─────────────────────────────────────────────
# Cell 5 · 执行
# ─────────────────────────────────────────────

# Step 1 · ASR
raw_asr = run_asr_from_srt(audio_file, srt_file)

# Step 2+3 · 弹幕校对（可选）
if dm_path:
    clean_dm     = run_danmaku_cleaning(dm_path)
    proofed_data = run_smart_proofread(client, raw_asr, clean_dm, my_bg)
else:
    print("⏭️  已跳过弹幕校对")
    proofed_data = [{"start": s["start"], "end": s["end"], "ja": s["text"]} for s in raw_asr]

# Step 4 · 翻译
final_data = run_parallel_translation(client, proofed_data)

# 写出双语 SRT
output_srt = f"{Path(audio_file).stem}_bilingual.srt"
write_bilingual_srt(final_data, output_srt)

print(f"\n✅ 全部完成！Token 消耗估算：{usage_stats.total_tokens:,}")
files.download(output_srt)